# 开课吧NLP-Project01项目概述

<font color=black size=2>**项目负责人：**蒋晓天</font>

### 项目目标

这次项目的<u>**目标**</u>是建立一个在<u>**网页端**</u>使用的<u>**自动摘要应用**</u>

### 项目流程细节

- 1 在本地用python完成自动摘要应用模型 `f(content, title) -> summarization `
    - 1.1 输入句子的向量化
        - 1.1.1 完成词向量
            - 维基百科中文语料和汉语新闻语料库的处理
                - 下载地址：[维基百科中文语料](https://dumps.wikimedia.org/zhwiki/20190720/zhwiki-20190720-pages-articles.xml.bz2)及其专门的[提取工具](https://github.com/attardi/wikiextractor)，[汉语新闻语料库](https://github.com/Computing-Intelligence/datasource/blob/master/export_sql_1558435.zip)
                - 数据提取并清洗
            - 使用Gensim训练词向量
            - 测试词向量效果
                - 语义相似性
                - 语义线性关系
                - 词向量的可视化(用[t-sne](https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne))
        - 1.1.2 基于上一步的词向量用SIF对句子进行向量化
            - 实现普林斯顿大学提出的[SIF算法](https://openreview.net/pdf?id=SyK00v5xx)
            - 对3类内容进行句子向量化
                - 新闻中的每一句话计算向量：$ V_{S_j} $
                - 整篇新闻作为一个完整长句计算向量：$ V_c $
                - 标题作为一个长居计算向量：$ V_t $
    - 1.2 获得句子与文章和标题的相关度并输出摘要
        - 我们已经获得了每个句子的向量$V_{S_0},V_{S_1},V_{S_2}, ... V_{S_n}$，每个文章的标题向量$V_t$，每个文章全文的向量$C_t$，接着建立一个模型$ f(V_{S_i}, V_t, V_c) $，对每一个$ V_{S_i} $输出一个(0, 1)的值，代表这句话与全文的相关度$ C_i $
        - 对 $ C_i $进行排序，取出$Top_n$个句子组成摘要
            - 需要说明的是，如果直接使用上述的结果，会出现一个问题，由于我们是根据句子和全文的语义相关性来排序的，因此取出来的句子很可能互相之间不是很通顺。要解决这个问题，需要做两件事情
                - 1 有些句子虽然和全文以及标题的语义相关性较低，但是却位于$C_i$比较高的句子前后，那么我们应该分配一些权重给这些句子以达到句义连贯性问题，所以这里使用KNN思想，在求得所有$C_i$后，对所有$C_i$和其附近的若干$C_j$进行加权求和获得新的$C_i'$
                - 2 在排序后，依旧按照句子在文章里的顺序输出句子
    - 1.3 模型整合和参数调整
        - 1 将之前的所有函数整合成一个，接受的输入为`title`，`content`和输出的句子数量`n`，以及其他超参
        - 2 可调参数分别为
            - sif的a，可选值为`1e-3`和`1e-5`
            - weight，在计算相关度时，标题和全文内容的比重，取值范围是\[0-1\]
            - KNN平滑时选择的范围和权重(经过测试发现平均权重和非平均权重的差别不是很大，因此项目选择了平均法，并且范围为5)
- 2 将模型上传到开课吧服务器并且正常运行
    - 1 用`flask`搭建前端
    - 2 将模型以及前端文件通过FileZilla上传到服务器
    - 3 创建新的虚拟环境，并用nohup让flask app保持运行状态
    - 4 测试

### 项目文件概述

>`articleSummary.py`主程序，初始化模型，并启动flask<br>
`summaryModel.py`定义了模型，以及编写接口用来获取文章摘要<br>
~~`model_191115_1`等3个文件均为训练完的word2vec模型文件~~<br>
`templates`文件夹，包含了3个html文档，分别为`base.html`，`input.html`，`result.html`。后两者继承了`base.html`的模版，`input.html`为输入数据页面，`result.html`为显示结果页面

### 关键技术点和解决方案

##### 维基百科的内容提取

>1. 下载[维基百科中文语料](https://dumps.wikimedia.org/zhwiki/20190720/zhwiki-20190720-pages-articles.xml.bz2)
2. 将[提取工具Wikietractor](https://github.com/attardi/wikiextractor) clone到本地
3. ~~按照Wikiextractor脚本`python setup.py install`~~(很多博客都写了这一条，但事实上不需要，直接运行主程序即可)
4. 运行`python3 WikiExtractor.py -b 500M -o target_dir zhwiki-20190720-pages-articles-multistream.xml.bz2`<br>
很多博客写的很多是`python Wiki...`而不是`python3 Wiki...`。一开始我也用`python`但是无法提取内容，然后想了一下试了试`python3`，结果就可以了，之后就等待提取结束即可(大约1hr)

##### 维基百科里简体和繁体混乱

>`pip install opencc-python-reimplemented`

In [1]:
import opencc
cc = opencc.OpenCC('t2s')
cc.convert('數學透過抽象化和邏輯推理的使用')

'数学透过抽象化和逻辑推理的使用'

##### 去掉所有标点符号和换行符

In [4]:
import re
punctuation = '！？｡。，＂＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘\'‛“”„‟…‧﹏' + '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~·ʔ•'
test = '此一希腊语被亚里士多德拿来指「万物皆数」的概念。\n\n汉字表示的「数学」一词大约产生于中国宋元时期。多指象数之学，但有时也含有今天上的数学意义，例如，秦九韶的《数学九章》（《永乐大典》记，即《数书九章》也被宋代周密所著的《癸辛杂识》记为《数学大略》）、《数学通轨》（明代柯尚迁著）、《数学钥》（清代杜知耕著）、《数学拾遗》（清代丁取忠撰）。'
re.sub('[{}]'.format(punctuation+'\n'), ' ', test)

'此一希腊语被亚里士多德拿来指 万物皆数 的概念   汉字表示的 数学 一词大约产生于中国宋元时期 多指象数之学 但有时也含有今天上的数学意义 例如 秦九韶的 数学九章   永乐大典 记 即 数书九章 也被宋代周密所著的 癸辛杂识 记为 数学大略    数学通轨  明代柯尚迁著   数学钥  清代杜知耕著   数学拾遗  清代丁取忠撰  '

##### 读取汉语新闻语料库时出现编码问题

>`utf-8`和`gbk`都无法读取所有字符，使用编码更广的`gb18030`即可解决

##### 汉语新闻语料库处理时出现字符串相关问题

>在清洗数据时出现`TypeError: expected string or bytes-like object`<br>
原因是直接读取的新闻content和title并不全是string，因此在读取后需要用`str()`将所有内容转化成string后再进行字符串清洗工作

##### matplotlib不正确显示中文

>添加`plt.rcParams['font.sans-serif'] = ['SimHei'] #指定默认字体`和<br>`plt.rcParams['axes.unicode_minus'] = False #解决保存图像是负号'-'显示为方块的问题`即可

##### 计算词频

>`word2vec`model可以调取`count`：`model.vw.vocab[word].count`。这样的话遍历`model.vw.vocab.keys()`就能统计所有词的计数，进而得到词频

##### 输入的单词不在训练的模型中

>词向量为0

##### 分句时如何保留标点符号

>由于最后需要输出的是带标点符号的摘要，因此如果直接用标点符号来分句，那么每个句子都没有了句尾标点符号<br>
这里采用的方式是，先在分句的标点符号后面插入特殊的标志(比如`'@#}~'`等)，然后根据标志来分局，这样就能在保留标点符号的情况下进行分句

##### 如何同时匹配'\\\\n'和'\n\r'

>用`\\\\n|[\n\r]`。关键点在于`|`旁边不能有空格，否则无法匹配

##### 参数选择和测试

>由于这个项目的输出是一段摘要，摘要是否合理需要主观去判断，因此很难进行标准的测试流程<br>
我在项目中选择了5篇长新闻(1k字数左右)对参数进行测试。最后主观感觉在参数分别为：`a=1e-5`, `weigt=0.4`(title占0.6，content占0.4)时效果最好


##### flask

>由于完全没有前端的基础，flask这一块卡了特别久，从url定位到html编写到添加css再上上线花的时间比写模型还久。最后结果也不太如意，没有写完error.html，于是没办法捕捉error后再进行跳转到主页面。在ddl后自己课后会补上。